In [1]:
import skimage.io
import dask.array as da
import dask
from pathlib import Path
import napari
from natsort import natsorted
import numpy as np
from napari_flim_phasor_calculator._reader import get_current_tz, read_ptu_data_2D, read_ptu_3D_data

Define read 2d and 3d functions

In [2]:

# folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\embryo_FLIM_data\raw_data_embryo_stack_3tps_43pl_2ch\embryo_43pl_2ch_3tps"
folder_path = r"C:\Users\mazo260d\Desktop\Conni_BiA_PoL\stack_smaller"


folder_path = Path(folder_path)

Get max slice shape

In [3]:
# Read all slices to get slice max shape and dtype (not ideal but for now it is OK)
slice_shape_list = []
for file_path in folder_path.iterdir():
    if file_path.suffix == '.ptu':
        image_2D = read_ptu_data_2D(file_path)
        slice_shape_list.append(image_2D.shape)
slice_max_shape = max(slice_shape_list)
print('last_slice_shape = ', image_2D.shape,'image_dtype = ', image_2D.dtype)
print('max_slice_shape = ', slice_max_shape)

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardwa

Get max z slices and max t timepoints from filepaths

In [4]:
file_paths = [file_path for file_path in folder_path.iterdir() if file_path.suffix == '.ptu']

# Get max z slices by reading all file names
def get_max_zslices(file_paths):
    max_z = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.ptu'])[1]
    if max_z is None:
        return 1
    return max_z
def get_max_time(file_paths):
    max_time = max([get_current_tz(file_path) for file_path in file_paths if file_path.suffix == '.ptu'])[0]
    if max_time is None:
        return 1
    return max_time

max_z = get_max_zslices(file_paths)
print('max_z slices = ', max_z)
max_t = get_max_time(file_paths)
print('max_time = ', max_t)

max_z slices =  43
max_time =  1


Get stack max shape

In [5]:
image_stack_shape = (*slice_max_shape[:-2], max_z, *slice_max_shape[-2:])
image_stack_shape


(2, 136, 43, 256, 256)

Make dask array

In [6]:
t_path_list = []
z_path_list = []
file_paths = natsorted(file_paths)
previous_t = 1
for file_path in file_paths:
    if file_path.suffix == '.ptu':
        current_t, current_z = get_current_tz(file_path)
        print(current_t, current_z)
        if current_t is not None:
            if current_t > previous_t:
                t_path_list.append(z_path_list)
                z_path_list = []
                previous_t = current_t
            z_path_list.append(file_path)

# If no timepoints, z+path_list is file_paths
if current_t is None:
    z_path_list = file_paths

# Append last timepoint
t_path_list.append(z_path_list)


None 1
None 2
None 3
None 4
None 5
None 6
None 7
None 8
None 9
None 10
None 11
None 12
None 13
None 14
None 15
None 16
None 17
None 18
None 19
None 20
None 21
None 22
None 23
None 24
None 25
None 26
None 27
None 28
None 29
None 30
None 31
None 32
None 33
None 34
None 35
None 36
None 37
None 38
None 39
None 40
None 41
None 42
None 43


In [7]:
z_path_list

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z1.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z2.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z3.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z4.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z5.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z6.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z7.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z8.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z9.ptu'),
 WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z10.ptu'),
 WindowsP

In [8]:
for z_path_list in t_path_list:
    print(z_path_list)

[WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z1.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z2.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z3.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z4.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z5.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z6.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z7.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z8.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z9.ptu'), WindowsPath('C:/Users/mazo260d/Desktop/Conni_BiA_PoL/stack_smaller/embryo_43pl_2ch_3tps_z10.ptu'), WindowsPath('C:/Us

In [9]:
read_ptu_data_3D_delayed = dask.delayed(read_ptu_3D_data, pure=True)

lazy_images = [read_ptu_data_3D_delayed(z_path_list, image_slice_shape=slice_max_shape) for z_path_list in t_path_list]

arrays = [da.from_delayed(lazy_image,           # Construct a small Dask array
                          dtype=image_2D.dtype,   # for every lazy value
                          shape=image_stack_shape)
          for lazy_image in lazy_images]

stack = da.stack(arrays, axis=0)                # Stack all small Dask arrays into one
print('stack shape and chunks = ', stack.shape, '\n', stack.chunks)

stack shape and chunks =  (1, 2, 136, 43, 256, 256) 
 ((1,), (2,), (136,), (43,), (256,), (256,))


In [10]:
stack = da.moveaxis(stack, 0, -4)

In [11]:
stack

dask.array<transpose, shape=(2, 136, 1, 43, 256, 256), dtype=uint16, chunksize=(2, 136, 1, 43, 256, 256), chunktype=numpy.ndarray>

In [13]:
stack.chunksize

(2, 136, 1, 43, 256, 256)

In [16]:
np.cumprod(stack.chunksize)

array([        2,       272,       272,     11696,   2994176, 766509056])

In [14]:
stack.itemsize

2

In [12]:
chunk_size_MBytes = np.cumprod(stack.chunksize)[-1] * stack.itemsize / 1e6 #MB per chunk
chunk_size_MBytes

1533.018112

In [13]:
if chunk_size_MBytes > 1e3: # if larger than 1GB (1000MB)
    stack = stack.rechunk({-3: 'auto'}, block_size_limit=1e9)

In [14]:
stack

dask.array<rechunk-merge, shape=(2, 136, 1, 43, 256, 256), dtype=uint16, chunksize=(2, 136, 1, 28, 256, 256), chunktype=numpy.ndarray>

In [15]:
viewer = napari.Viewer()

In [16]:
viewer.add_image(stack, channel_axis=0)

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardware: PicoHarpT3
Raw Data has been Read!

TCSPC Hardwa

[<Image layer 'Image' at 0x26fd0f3a2b0>,
 <Image layer 'Image [1]' at 0x26fca3ad3a0>]